In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/eclipse-bert-cnn-model/tokenization.py
/kaggle/input/eclipse-bert-cnn-model/eclipse_bert_cnn/saved_model.pb
/kaggle/input/eclipse-bert-cnn-model/eclipse_bert_cnn/keras_metadata.pb
/kaggle/input/eclipse-bert-cnn-model/eclipse_bert_cnn/assets/vocab.txt
/kaggle/input/eclipse-bert-cnn-model/eclipse_bert_cnn/variables/variables.index
/kaggle/input/eclipse-bert-cnn-model/eclipse_bert_cnn/variables/variables.data-00000-of-00001
/kaggle/input/netbeans-preprocessed5/netbeans_preprocessed_os.csv


In [2]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/tools/tokenization.py

In [3]:
!pip install sentencepiece

In [4]:
!pip install tensorflow_hub

In [5]:
!pip install torch

In [6]:
import nltk
nltk.download('punkt')
nltk.download("stopwords")
nltk.download("words")
from nltk.corpus import stopwords
from nltk.corpus import words
# words = set(nltk.corpus.words.words())

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /usr/share/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [7]:
import pandas as pd
import numpy as np
from random import seed
from random import randint
import random
import string
import re
import matplotlib.pyplot as plt
# from google.colab import drive
# drive.mount('/content/drive')

In [8]:
from tensorflow.keras.layers import Dense, Input, Reshape, Conv1D, Conv2D, BatchNormalization, MaxPooling1D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, Sequential, load_model
# from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
import tensorflow as tf
from sklearn.model_selection import train_test_split

# from bert import bert_tokenization
import tokenization
from sklearn.model_selection import StratifiedKFold

In [9]:
gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
session = tf.compat.v1.InteractiveSession(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

2022-03-22 16:08:40.213102: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-22 16:08:40.253319: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-22 16:08:40.336553: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-22 16:08:40.337536: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [10]:
seed(1)
stop_words = stopwords.words('english')
words=words.words()

In [11]:
# train_indices = []
# test_indices = []

# with open('../input/netbeans-train-test1/nb_train1.txt') as f:
#     train_indices = f.readlines()

# with open('../input/netbeans-train-test1/nb_test1.txt') as f:
#     test_indices = f.readlines()

In [12]:
# train_indices = [int(tr.split('\n')[0]) for tr in train_indices]

In [13]:
# len(train_indices)

In [14]:
# test_indices  = [int(te.split('\n')[0]) for te in test_indices]

In [15]:
# len(test_indices)

In [16]:
dataset = pd.read_csv('../input/netbeans-preprocessed5/netbeans_preprocessed_os.csv')
dataset

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,bug_id,description1,short_desc1,dup_id,resolution,product,component,description2,short_desc2,duplicate
0,172972,188204,Product Version = NetBeans IDE 6.9 (Build 2010...,Provide way to distinguish between files with ...,178603,DUPLICATE,editor,Actions/Menu/Toolbar,164370 In symfony(and probably other framew...,164370 show full pathname of opened files,1
1,48515,52089,The access to the cvs server in advanced mode ...,Save of standards fails,44124,DUPLICATE,versioncontrol,CVS,"40724 I have some CVS mounts, e.g. nb_all. ...",40724 CVS advanced option defaults not pers...,1
2,89709,96112,"I'm using 5.5 (build 200610171010), Java 1.6.0...",Dump on Refactor->Rename,99537,DUPLICATE,obsolete,visualweb,"92876 ""Rename"" on inner protected class alw...","92876 ""Rename"" function error",1
3,44732,48203,If you add a directory to favorites and then\n...,Delete action in Favorites is unnerving,46986,DUPLICATE,platform,Favorites,"43544 [nb_dev](200408051800), [jdk1.5.0](b6...",43544 Delete favorite folder/file doesn't m...,1
4,8531,11215,Number of build where a bug happened - [36]\nJ...,editor freezes upon file change and reload fro...,10961,DUPLICATE,platform,-- Other --,8278 [],8278 Deadlock in CloneableEditorSupport,1
...,...,...,...,...,...,...,...,...,...,...,...
180315,72853,77716,After delete test files is not possible create...,J2MEUNIT - After delete test files is not poss...,[],FIXED,javame,JUnit,"In my maven projects, I have a library depende...",Editor falsely applies error badges in source ...,0
180316,171492,186537,select * from my table where mycolumn like '01...,New line before LIMIT of OFFSET is not interpr...,[],FIXED,db,SQL Editor,----------------------------------------------...,Unexpected Exception: SAXException at startup,0
180317,165744,180162,Build: NetBeans IDE Dev (Build 100202-1313c761...,NullPointerException at com.sun.tools.javac.mo...,[],FIXED,java,Compiler,[ JDK VERSION : J2SE 1.4.2_02 ]\n\nHello\nusin...,[36cat] the same description for two entries,0
180318,207541,227785,Build: NetBeans IDE 7.3 (Build 201302132200)\n...,org.netbeans.modules.subversion.client.SvnClie...,[],FIXED,versioncontrol,Subversion,Product Version = NetBeans IDE Dev (Build nbms...,Problems parsing PHPDoc,0


In [17]:
dataset.dropna(subset = ["description1"], inplace=True)
dataset.dropna(subset = ["description2"], inplace=True)

In [18]:
for index, row in dataset.iterrows():
    doc1 = 'Product:' + row['product'] + '.Component:' + row['component'] + '.Description:' + row['description1'] #+ '.Summary:' + row['short_desc1']
    doc2 = 'Product:' + row['product'] + '.Component:' + row['component'] + '.Description:' + row['description2'] #+ '.Summary:' + row['short_desc2']
    dataset.loc[index, 'doc1'] = doc1
    dataset.loc[index, 'doc2'] = doc2

In [19]:
for index, row in dataset.iterrows():
    desc = row['doc1'] + row['doc2']
    dataset.loc[index, 'description'] = desc

In [20]:
def reset_column_names():
  dataset.drop('description', axis=1, inplace=True)
#   dataset.drop('short_desc1', axis=1, inplace=True)
#   dataset.drop('description2', axis=1, inplace=True)
#   dataset.drop('short_desc2', axis=1, inplace=True)

  dataset.rename(columns={'description_clean':'description'}, inplace=True) #,'short_desc1_clean':'short_desc1','description2_clean':'description2','short_desc2_clean':'short_desc2'

In [21]:
def preprocess_text_content(text):
    text = text.lower()
#     cleaned_text = re.sub(r"""
#                [,.;:@#?!&$*+-=_%<>\/\[\]\(\)\{\}\"\'\n]+  # Accept one or more copies of punctuation
#                \ *           # plus zero or more copies of a space,
#                """,
#                " ",          # and replace it with a single space
#                text, flags=re.VERBOSE)
#     cleaned_text = re.sub(r"https?:\/\/[A-Za-z0-9.\/?&#+*+-=_%]+", " ",text) #replace urls
    cleaned_text = re.sub(r" +", " ", text) #remove extra white spaces
#     cleaned_text = re.sub(r"[d|D]escription", "", cleaned_text)
    
    return cleaned_text

In [22]:
dataset['description_clean']= dataset['description'].apply(lambda x:preprocess_text_content(x))
# dataset['short_desc1_clean']= dataset['short_desc1'].apply(lambda x:preprocess_text(x))
# dataset['description2_clean']= dataset['description2'].apply(lambda x:preprocess_text(x))
# dataset['short_desc2_clean']= dataset['short_desc2'].apply(lambda x:preprocess_text(x))
dataset.head()

,Unnamed: 0,bug_id,description1,short_desc1,dup_id,resolution,product,component,description2,short_desc2,duplicate,doc1,doc2,description,description_clean
0,172972,188204,Product Version = NetBeans IDE 6.9 (Build 2010...,Provide way to distinguish between files with ...,178603,DUPLICATE,editor,Actions/Menu/Toolbar,164370 In symfony(and probably other framew...,164370 show full pathname of opened files,1,Product:editor.Component:Actions/Menu/Toolbar....,Product:editor.Component:Actions/Menu/Toolbar....,Product:editor.Component:Actions/Menu/Toolbar....,product:editor.component:actions/menu/toolbar....
1,48515,52089,The access to the cvs server in advanced mode ...,Save of standards fails,44124,DUPLICATE,versioncontrol,CVS,"40724 I have some CVS mounts, e.g. nb_all. ...",40724 CVS advanced option defaults not pers...,1,Product:versioncontrol.Component:CVS.Descripti...,Product:versioncontrol.Component:CVS.Descripti...,Product:versioncontrol.Component:CVS.Descripti...,product:versioncontrol.component:cvs.descripti...
2,89709,96112,"I'm using 5.5 (build 200610171010), Java 1.6.0...",Dump on Refactor->Rename,99537,DUPLICATE,obsolete,visualweb,"92876 ""Rename"" on inner protected class alw...","92876 ""Rename"" function error",1,Product:obsolete.Component:visualweb.Descripti...,Product:obsolete.Component:visualweb.Descripti...,Product:obsolete.Component:visualweb.Descripti...,product:obsolete.component:visualweb.descripti...
3,44732,48203,If you add a directory to favorites and then\n...,Delete action in Favorites is unnerving,46986,DUPLICATE,platform,Favorites,"43544 [nb_dev](200408051800), [jdk1.5.0](b6...",43544 Delete favorite folder/file doesn't m...,1,Product:platform.Component:Favorites.Descripti...,Product:platform.Component:Favorites.Descripti...,Product:platform.Component:Favorites.Descripti...,product:platform.component:favorites.descripti...
4,8531,11215,Number of build where a bug happened - [36]\nJ...,editor freezes upon file change and reload fro...,10961,DUPLICATE,platform,-- Other --,8278 [],8278 Deadlock in CloneableEditorSupport,1,Product:platform.Component:-- Other --.Descrip...,Product:platform.Component:-- Other --.Descrip...,Product:platform.Component:-- Other --.Descrip...,product:platform.component:-- other --.descrip...


In [23]:
reset_column_names()

In [24]:
# def preprocess_text(text,tokenizer):
# #     print(text)
#     #tokenize the text
#     text = tokenizer.tokenize(text)
#     #lowercase the text
#     text = [w.lower() for w in text]
#     #removal of stop word 
#     text = [w for w in text if w not in stop_words]
#     #removal of non-english words
#     text = [w for w in text if w in words or w.isalpha()]
    #stemming
#     text = [ ps.stem(w) for w in text]
    #remove extra white spaces
#     text = [ re.sub(r" +", " ", w) for w in text] 
#     cleaned_text = re.sub(r"""
#                [,.;:@#?!&$*+-=_%<>\/\[\]\(\)\{\}\"\'\n]+  # Accept one or more copies of punctuation
#                \ *           # plus zero or more copies of a space,
#                """,
#                " ",          # and replace it with a single space
#                text, flags=re.VERBOSE)
#     cleaned_text = re.sub(r"https?:\/\/[A-Za-z0-9.\/?&#+*+-=_%]+", " ",cleaned_text) #replace urls
    
#     cleaned_text = re.sub(r"[d|D]escription", "", cleaned_text)
    
#     return text

In [25]:
max_len = 0
lenArr = []
outliers = []

# q25, q75 = np.percentile(lenArr, [25, 75])
# bin_width = 2 * (q75 - q25) * len(lenArr) ** (-1/3)
# bins = round((lenArr.max() - lenArr.min()) / bin_width)

### Add tokens to the data make it BERT compatible
def bert_encode(texts, tokenizer, max_len=512):
    print('encode')
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
#         text = preprocess_text(text,tokenizer)
        text = tokenizer.tokenize(text)
#         print(text)
        text = [w for w in text if w.lower() not in stop_words]
    
#         length = len(text)
#         if(length <1000):
#             lenArr.append(length)
#     #     print(length)
#             if(max_len < length):
#               max_len = length
#         else:
#             outliers.append(length)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
#     print(len(lenArr))
#     # lenArr=list(filter(lambda a: a != 3395, lenArr))
#     # print(lenArr)
#     print("avg", sum(lenArr)/len(lenArr))
#     print("outliers", outliers)
#     print("outliers", len(outliers))
#     plt.hist(lenArr, density=True, bins=30)
#     max_len

    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [26]:
def build_model(bert_layer, max_len=512):
    print('build model')
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
    
#     embedding = bert_layer(
#         input_word_ids, token_type_ids=segment_ids, attention_mask=input_mask
#     )[0]

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :] 
#     print(clf_output)
   
    clf_output = Reshape((1024,1))(clf_output)
    print(clf_output)
    
    x = Conv1D(32, 3, activation='relu', padding='same')(clf_output)
    x = BatchNormalization()(x)
    x = MaxPooling1D()(x)
    x = Conv1D(32, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D()(x)
#     x = Flatten()(x)
#     out = Dense(1, activation='sigmoid')(x)
    out = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    
    return model

In [27]:
train, test = train_test_split(dataset, test_size=0.3)

In [28]:
FullTokenizer = tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1", trainable=True)

2022-03-22 16:49:04.521566: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-22 16:49:04.522436: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-22 16:49:04.523079: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-22 16:49:04.523951: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-22 16:49:04.524603: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [29]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
lower_cased = bert_layer.resolved_object.do_lower_case.numpy()

tokenizer = FullTokenizer(vocab_file, lower_cased)

In [30]:
# X = dataset['description']
# Y = np.array(dataset.duplicate.values, dtype='int')

In [31]:
train_input = bert_encode(train["description"], tokenizer, max_len=160)
train_labels = train.duplicate.values

encode


In [32]:
# model = build_model(bert_layer, max_len=160)
# model.summary()
model = load_model("../input/eclipse-bert-cnn-model/eclipse_bert_cnn")
# Fit the model
model.fit(train_input, train_labels, validation_split=0.3, epochs=4, batch_size=15)

2022-03-22 17:27:27.701619: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/4


2022-03-22 17:28:07.028707: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


5885/5885 [==============================] - 6032s 1s/step - loss: 0.2450 - accuracy: 0.8691 - precision: 0.8532 - recall: 0.8920 - val_loss: 0.2079 - val_accuracy: 0.8968 - val_precision: 0.9121 - val_recall: 0.8771
Epoch 2/4
5885/5885 [==============================] - 5975s 1s/step - loss: 0.1327 - accuracy: 0.9436 - precision: 0.9382 - recall: 0.9499 - val_loss: 0.1425 - val_accuracy: 0.9385 - val_precision: 0.9218 - val_recall: 0.9575
Epoch 3/4
5885/5885 [==============================] - 5978s 1s/step - loss: 0.0452 - accuracy: 0.9849 - precision: 0.9835 - recall: 0.9864 - val_loss: 0.1238 - val_accuracy: 0.9598 - val_precision: 0.9637 - val_recall: 0.9550
Epoch 4/4
5885/5885 [==============================] - 5970s 1s/step - loss: 0.0232 - accuracy: 0.9938 - precision: 0.9935 - recall: 0.9942 - val_loss: 0.1561 - val_accuracy: 0.9515 - val_precision: 0.9326 - val_recall: 0.9728


In [33]:
model.save("eclipse_netbeans_cnn")

2022-03-23 00:07:31.588472: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [34]:
test_input = bert_encode(test["description"], tokenizer, max_len=160)
test_labels = test.duplicate.values

encode


In [35]:
scores = model.evaluate(test_input, test_labels, verbose=0)
f1score = (2 * scores[2]*100 * scores[3]*100)/(scores[2]*100 + scores[3]*100)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print("%s: %.2f%%" % (model.metrics_names[2], scores[2]*100))
print("%s: %.2f%%" % (model.metrics_names[3], scores[3]*100))
print("f1 score: ", (f1score))

accuracy: 95.11%
precision: 93.27%
recall: 97.27%
f1 score:  95.23208365589753


In [36]:
# train_input = bert_encode(dataset['description'], tokenizer, max_len=160)
# test_input = bert_encode(test['description'], tokenizer, max_len=160)
# train_labels = np.array(train.duplicate.values, dtype='int')
# test_labels = np.array(test.duplicate.values, dtype='int')

In [37]:
# max_len = 0
# lenArr = []
# outliers = []

# for index, row in dataset.iterrows():
#     # print(train.loc[[index]]['description'])
#     length = max([len(s.split()) for s in dataset.loc[[index]]['description']])
#     if(length <1000):
#         lenArr.append(length)
# #     print(length)
#         if(max_len < length):
#           max_len = length
#     else:
#         outliers.append(length)

# # q25, q75 = np.percentile(lenArr, [25, 75])
# # bin_width = 2 * (q75 - q25) * len(lenArr) ** (-1/3)
# # bins = round((lenArr.max() - lenArr.min()) / bin_width)
# print(len(lenArr))
# # lenArr=list(filter(lambda a: a != 3395, lenArr))
# # print(lenArr)
# print("avg", sum(lenArr)/len(lenArr))
# print("outliers", outliers)
# print("outliers", len(outliers))
# plt.hist(lenArr, density=True, bins=30)
# max_len

In [38]:
# model = build_model(bert_layer, max_len=160)
# model.summary()

In [39]:
# train_history = model.fit(
#     train_input, train_labels,
#     validation_split=0.3,
#     epochs=5,
#     batch_size=50
# )

In [40]:
# evaluation_results = model.evaluate(test_input,test_labels,return_dict=True)
# evaluation_results

In [41]:
# X = dataset['description']
# Y = np.array(dataset.duplicate.values, dtype='int')

# kfold = StratifiedKFold()
# cvscores = []
# precision_scores = []
# recall_scores = []
# f1_scores = []
    
# for train, test in kfold.split(X, Y):
#     # Prepare data
#     train_input = bert_encode(X.iloc[train], tokenizer, max_len=160)
#     test_input = bert_encode(X.iloc[test], tokenizer, max_len=160)
#     train_labels = Y[train]
#     test_labels = Y[test]
    
#     model = build_model(bert_layer, max_len=160)
#     model.summary()

#     # Fit the model
#     model.fit(train_input,train_labels, epochs=2, batch_size=15)

#     # Evaluate the model
#     scores = model.evaluate(test_input, test_labels, verbose=0)
#     f1score = (2 * scores[2]*100 * scores[3]*100)/(scores[2]*100 + scores[3]*100)
#     print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
#     print("%s: %.2f%%" % (model.metrics_names[2], scores[2]*100))
#     print("%s: %.2f%%" % (model.metrics_names[3], scores[3]*100))
#     print("f1 score: ", (f1score))
#     cvscores.append(scores[1] * 100)
#     precision_scores.append(scores[2] * 100)
#     recall_scores.append(scores[3] * 100)
#     f1_scores.append(f1score)
    

# print("accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
# print("precision: %.2f%% (+/- %.2f%%)" % (np.mean(precision_scores), np.std(precision_scores)))
# print("recall: %.2f%% (+/- %.2f%%)" % (np.mean(recall_scores), np.std(recall_scores)))
# print("f1 score: %.2f%% (+/- %.2f%%)" % (np.mean(f1_scores), np.std(f1_scores)))